<a href="https://colab.research.google.com/github/rileydixon6/machine_learning_practice/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
# Load file data
text = open("/content/doyle.txt", 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1737118 characters


In [ ]:
# Verify the first part of our data
print(text[:200])

﻿

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: November 29, 2002 [EBook #1661]
Last Updated: May 20, 2019

Language: English

Character set encoding:


In [ ]:
# Now we'll get a list of the unique characters in the file. This will form the
# vocabulary of our network. There may be some characters we want to remove from this 
# set as we refine the network.
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

98 unique characters
['\t', '\n', '\r', ' ', '!', '#', '&', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', 'º', '½', 'à', 'â', 'æ', 'è', 'é', 'î', 'ô', 'œ', '—', '‘', '’', '“', '”', '\ufeff']


In [ ]:
# Encoding encode these characters into numbers so we can use them
# with our neural network, then we'll create some mappings between the characters
# and their numeric representations
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)

# Here's a little helper function that we can use to turn a sequence of ids
# back into a string:
# turn them into a string:
def text_from_ids(ids):
  joinedTensor = tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
  return joinedTensor.numpy().decode("utf-8")

In [ ]:
# Now we'll verify that they work, by getting the code for "A", and then looking
# that up in reverse
testids = ids_from_chars(["T", "r", "u", "t", "h"])
testids

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([47, 74, 77, 76, 64])>

In [ ]:
chars_from_ids(testids)

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'T', b'r', b'u', b't', b'h'], dtype=object)>

In [ ]:
testString = text_from_ids( testids )
testString

'Truth'

In [ ]:
# Create a stream of encoded integers from our text
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1737118,), dtype=int64, numpy=array([99,  4,  3, ..., 79, 70, 14])>

In [ ]:
# Convert that into a tensorflow dataset
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
# Batch these sequences up into chunks for our training
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# This function will generate our sequence pairs:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Call the function for every sequence in our list to create a new dataset
# of input->target pairs
dataset = sequences.map(split_input_target)

In [ ]:
# Verify our sequences
for input_example, target_example in  dataset.take(1):
    print("Input: ", text_from_ids(input_example))
    print("--------")
    print("Target: ", text_from_ids(target_example))

Input:  ﻿

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: Novembe
--------
Target:  

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: November


In [ ]:
# Randomize the sequences so that we don't just memorize the books
# in the order they were written, then build a new streaming dataset from that.
# Using a streaming dataset allows us to pass the data to our network bit by bit,
# rather than keeping it all in memory. We'll set it to figure out how much data
# to prefetch in the background.

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Create our custom model. Given a sequence of characters, this
# model's job is to predict what character should come next.
class DoyleTextModel(tf.keras.Model):

  # This is our class constructor method, it will be executed when
  # we first create an instance of the class 
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    # Our model will have three layers:
    
    # 1. An embedding layer that handles the encoding of our vocabulary into
    #    a vector of values suitable for a neural network
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # 2. A GRU layer that handles the "memory" aspects of our RNN. If you're
    #    wondering why we use GRU instead of LSTM, and whether LSTM is better,
    #    take a look at this article: https://datascience.stackexchange.com/questions/14581/when-to-use-gru-over-lstm
    #    then consider trying out LSTM instead (or in addition to!)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)

    # 3. Our output layer that will give us a set of probabilities for each
    #    character in our vocabulary.
    self.dense = tf.keras.layers.Dense(vocab_size)

  # This function will be executed for each epoch of our training. Here
  # we will manually feed information from one layer of our network to the 
  # next.
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs

    # 1. Feed the inputs into the embedding layer, and tell it if we are
    #    training or predicting
    x = self.embedding(x, training=training)

    # 2. If we don't have any state in memory yet, get the initial random state
    #    from our GRUI layer.
    if states is None:
      states = self.gru.get_initial_state(x)
    
    # 3. Now, feed the vectorized input along with the current state of memory
    #    into the gru layer.
    x, states = self.gru(x, initial_state=states, training=training)

    # 4. Finally, pass the results on to the dense layer
    x = self.dense(x, training=training)

    # 5. Return the results
    if return_state:
      return x, states
    else: 
      return x

In [ ]:
# Create an instance of our model
vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

model = DoyleTextModel(vocab_size, embedding_dim, rnn_units)

In [ ]:
# Verify the output of our model is correct by running one sample through
# This will also compile the model for us. This step will take a bit.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 100) # (batch_size, sequence_length, vocab_size)


In [ ]:
# Now let's view the model summary
model.summary()

Model: "doyle_text_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  25600     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  102500    
Total params: 4,066,404
Trainable params: 4,066,404
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

history = model.fit(dataset, epochs=20)

Epoch 1/20
268/268 [==============================] - 16s 51ms/step - loss: 2.9758
Epoch 2/20
268/268 [==============================] - 15s 51ms/step - loss: 1.7003
Epoch 3/20
268/268 [==============================] - 15s 51ms/step - loss: 1.3906
Epoch 4/20
268/268 [==============================] - 15s 52ms/step - loss: 1.2361
Epoch 5/20
268/268 [==============================] - 15s 52ms/step - loss: 1.1440
Epoch 6/20
268/268 [==============================] - 15s 52ms/step - loss: 1.0787
Epoch 7/20
268/268 [==============================] - 15s 53ms/step - loss: 1.0178
Epoch 8/20
268/268 [==============================] - 15s 53ms/step - loss: 0.9644
Epoch 9/20
268/268 [==============================] - 15s 54ms/step - loss: 0.9044
Epoch 10/20
268/268 [==============================] - 15s 54ms/step - loss: 0.8477
Epoch 11/20
268/268 [==============================] - 15s 54ms/step - loss: 0.7933
Epoch 12/20
268/268 [==============================] - 16s 54ms/step - loss: 0.7336
E

In [ ]:
# Here's the code we'll use to sample for us. It has some extra steps to apply
# the temperature to the distribution, and to make sure we don't get empty
# characters in our text. Most importantly, it will keep track of our model
# state for us.

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Return the characters and model state.
    return chars_from_ids(predicted_ids), states


In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['Chapter 1'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))




Chapter 187 a
      day or two things in front of my loper, I was too seriously could on
      the matter, then; and the detection is did all the information in
      the edge of the father and over his extreme knowledges than ill usually
      formed to change in his mind, and that he only deeply
      elbow, and if he were realised, therefore, to cut your hair, Holmes, who
knew her in your power of regret,” said Holmes as we
      answered him to pain with him which he clarped down, he remembered the
story was ever searched, but there was something in his eyes claimbed most head by a sharp rattle
      rather susterious imperturbably. And this he remained with a
      prompt and his exercise, rather stone up, the three
overcoat, we should have the two days of the question. You will find it
      two o’clock when she knocked without a word be
driver here without petriting in rusing the shoulder. A
      hand, keen-faced young fellow, steeping in the same
      dight, however, wa


In [ ]:
class CustomTraining(DoyleTextModel):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(vocab_size, embedding_dim, rnn_units)

  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size,
    embedding_dim,
    rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=20)

Epoch 1/20
268/268 [==============================] - 31s 106ms/step - loss: 2.3762
Epoch 2/20
268/268 [==============================] - 29s 106ms/step - loss: 1.5951
Epoch 3/20
268/268 [==============================] - 29s 106ms/step - loss: 1.3373
Epoch 4/20
268/268 [==============================] - 29s 105ms/step - loss: 1.2101
Epoch 5/20
268/268 [==============================] - 29s 106ms/step - loss: 1.1294
Epoch 6/20
268/268 [==============================] - 29s 106ms/step - loss: 1.0664
Epoch 7/20
268/268 [==============================] - 29s 106ms/step - loss: 1.0110
Epoch 8/20
268/268 [==============================] - 29s 105ms/step - loss: 0.9564
Epoch 9/20
268/268 [==============================] - 29s 106ms/step - loss: 0.9024
Epoch 10/20
268/268 [==============================] - 29s 106ms/step - loss: 0.8464
Epoch 11/20
268/268 [==============================] - 29s 106ms/step - loss: 0.7905
Epoch 12/20
268/268 [==============================] - 29s 106ms/step - lo

In [ ]:
# Create an instance of the character generator
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

# Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
# as its starting text
states = None
next_char = tf.constant(['Chapter 1'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)

# Print the results formatted.
print(result[0].numpy().decode('utf-8'))

Chapter 13_s_, and now I beg that you
      will not find their stepfather for three years ago than no easy. I remember, and I
      made off with him. But Alec Fairbairn whose step
      How long they would not think there anywhere at the breakfast was clear, and
      made my secret with a persettern, and, walking towards the bell, and,
      married, without a careful examination mad with a
chill of room, but I call it for handy demonstressed by my
      promise, that he may have been terribly impatiently asking him from
      my dressing-gown was to find am some one about that Sides and
      weight on with her. So much the woman thought
what was the business over, and nervous was a fool and swiftly affection. We had
      really two entwers—of the landlory’s office, and
      puzzled now of the wall as they had fallen our resist and
      learned. In left by the fire as we entered.

      “It was all the wife, and my wife by
other woman, black face, which purchased it with 
